In [32]:
import numpy as np

np.set_printoptions(precision=2, suppress=True)

# Our movie ratings example makes the concepts very clear:
X = np.array([
    [5, 4, 1, 2],    # User1: loves action (first 2), dislikes romance (last 2)
    [4, 5, 2, 1],    # User2: same pattern
    [1, 2, 5, 4],    # User3: opposite - loves romance, dislikes action
    [2, 1, 4, 5],    # User4: same pattern as User3
])

# Core concept: We want to find A and B matrices that give us two things:
# 1. Good predictions (X ≈ XAB^T)
# 2. Meaningful similarities between items

# The paper compares two ways to find these matrices:
# Objective 1: min ||X - XAB^T||²_F + λ||AB^T||²_F
# Objective 2: min ||X - XAB^T||²_F + λ(||XA||²_F + ||B||²_F)

# Let's demonstrate the key problem using Objective 1:
U, S, V = np.linalg.svd(X)
k = 2  # We use 2 dimensions (think: action vs romance)
lambda_reg = 10

# Get our base A and B matrices
S_transformed = S / (1 + lambda_reg/(S**2))
A = V.T[:, :k] @ np.diag(np.sqrt(S_transformed[:k]))
B = V.T[:, :k] @ np.diag(np.sqrt(S_transformed[:k]))

# Now the paper's key point: we can scale A and B differently but get same predictions
D1 = np.diag([1, 1])        # Original scaling
D2 = np.diag([2, 0.5])      # Different scaling
D3 = np.diag([10, 0.1])     # Extreme scaling

def demonstrate_scaling(D):
    A_new = A @ D
    B_new = B @ np.linalg.inv(D)
    
    # 1. Show predictions stay exactly the same
    pred_original = X @ A @ B.T
    pred_new = X @ A_new @ B_new.T
    
    # 2. But similarities change dramatically
    from sklearn.metrics.pairwise import cosine_similarity
    sim_new = cosine_similarity(B_new)
    
    return pred_new, sim_new

print("=== Original Scaling ===")
pred1, sim1 = demonstrate_scaling(D1)
print("Predictions:")
print(pred1)
print("\nItem Similarities:")
print(sim1)

print("\n=== Different Scaling ===")
pred2, sim2 = demonstrate_scaling(D2)
print("Predictions (notice they're the same):")
print(pred2)
print("\nItem Similarities (notice they're different!):")
print(sim2)

# Show prediction difference is zero
print("\nMaximum difference in predictions:", np.abs(pred1 - pred2).max())

=== Original Scaling ===
Predictions:
[[40.71 40.71 26.62 26.62]
 [40.71 40.71 26.62 26.62]
 [26.62 26.62 40.71 40.71]
 [26.62 26.62 40.71 40.71]]

Item Similarities:
[[1.   1.   0.41 0.41]
 [1.   1.   0.41 0.41]
 [0.41 0.41 1.   1.  ]
 [0.41 0.41 1.   1.  ]]

=== Different Scaling ===
Predictions (notice they're the same):
[[40.71 40.71 26.62 26.62]
 [40.71 40.71 26.62 26.62]
 [26.62 26.62 40.71 40.71]
 [26.62 26.62 40.71 40.71]]

Item Similarities (notice they're different!):
[[ 1.    1.   -0.74 -0.74]
 [ 1.    1.   -0.74 -0.74]
 [-0.74 -0.74  1.    1.  ]
 [-0.74 -0.74  1.    1.  ]]

Maximum difference in predictions: 0.0
